In [ ]:
#import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import chardet
from datetime import datetime, timedelta
import locale
from sklearn.decomposition import PCA
from sklearn import preprocessing
import prince

In [ ]:
populationDF = pd.read_csv(r'FilteredData\Population.csv')

In [ ]:
hemodynamicDF = pd.read_csv(r"DataCSV\Advancerede hæmodynamiske mål.csv" , sep = ";")
hemodynamicDF

In [ ]:
hemodynamicDF = hemodynamicDF[hemodynamicDF['cpr'].isin(populationDF['CPR'])].reset_index(drop = True)
hemodynamicDF[hemodynamicDF['Målingsnavn'] == 'PAP (middel)']

In [ ]:
temps = hemodynamicDF[hemodynamicDF['Målingsnavn'].str.startswith('Temp', na=False)]

In [ ]:
temps['Målingsnavn'].value_counts()

In [ ]:
hemodynamicDF[hemodynamicDF['Målingsnavn'] == 'Art']

In [ ]:
keep = ['ABP Puls (fra A-kanyle)' , 
        'ABP mean inv BT' , 
        'ART Puls (fra A-kanyle)' , 
        'ART mean inv BT' , 
        'CVP (Middel)' , 
        'CVP (mean)' , 
        'Invasivt BT - ABP (mean)' , 
        'Invasivt BT - ART (mean)' , 
        'LAP (mean)' , 'LAP (middel)' , 
        'MAP' , 'PAP (middel)' , 
        'Puls' , 
        'ABP Puls (fra A-kanyle)' , 
        r"Puls (fra BT's måling)" , 
        'Puls (fra SAT-måler)', 
        'Temperatur', 
        'Temp.kilde',
        'Temperatur 1',
        'Temperatur 3',
        'Temperatur, højre',
        'Temperatur 2',
        'Temperatur, venstre',
        'Temperatur 4',
        ]

In [ ]:
check = hemodynamicDF['Målingsnavn'].value_counts()

In [ ]:
hemodynamicKeptDF = hemodynamicDF[hemodynamicDF['Målingsnavn'].isin(keep)].drop(columns = ['Enhed' , 'Skema navn']).reset_index(drop = True)
hemodynamicKeptDF

In [ ]:
hemoDic = {'cpr'            : 'CPR' ,
           'Målingsdatotid' : 'MeasurementTime' ,
           'Målingsnavn'    : 'MeasurementName' ,
           'Værdi'          : 'Value'}
hemodynamicKeptDF = hemodynamicKeptDF.rename(columns = hemoDic)

In [ ]:
print(len(list(set(hemodynamicKeptDF['CPR']))))
hemodynamicKeptDF.isna().sum()

In [ ]:
hemodynamicKeptDF['Value'] = hemodynamicKeptDF['Value'].astype(float)
hemodynamicKeptDF.Value.dtype

In [ ]:
hemodynamicKeptDF = hemodynamicKeptDF[hemodynamicKeptDF['Value'].notna()]
hemodynamicKeptDF = hemodynamicKeptDF[hemodynamicKeptDF['Value'] > 0]
hemodynamicKeptDF = hemodynamicKeptDF[hemodynamicKeptDF['Value'] < 300].reset_index(drop = True)
hemodynamicKeptDF

In [ ]:
hemoDic = {'ABP Puls (fra A-kanyle)'  : 'MAP', 
           'ABP mean inv BT'          : 'MAP', 
           'ART Puls (fra A-kanyle)'  : 'MAP', 
           'ART mean inv BT'          : 'MAP', 
           'CVP (Middel)'             : 'CVP', 
           'CVP (mean)'               : 'CVP', 
           'Invasivt BT - ABP (mean)' : 'MAP', 
           'Invasivt BT - ART (mean)' : 'MAP', 
           'LAP (mean)'               : 'LAP', 
           'LAP (middel)'             : 'LAP', 
           'MAP'                      : 'MAP', 
           'PAP (middel)'             : 'PAP',
           'Puls'                     : 'Pulse', 
           'ABP Puls (fra A-kanyle)'  : 'Pulse', 
           r"Puls (fra BT's måling)"  : 'Pulse', 
           'Puls (fra SAT-måler)'     : 'Pulse'#,
           #'Temperatur'               : 'Temperature', 
           #'Temp.kilde'               : 'Temperature',
           #'Temperatur 1'             : 'Temperature',
           #'Temperatur 3'             : 'Temperature',
           #'Temperatur, højre'        : 'Temperature',
           #'Temperatur 2'             : 'Temperature',
           #'Temperatur, venstre'      : 'Temperature',
           #'Temperatur 4'             : 'Temperature',
           }

In [ ]:
hemodynamicKeptDF = hemodynamicKeptDF.replace({'MeasurementName' : hemoDic})
hemodynamicKeptDF['MeasurementName'].value_counts()

In [ ]:
hemodynamicKeptDF = hemodynamicKeptDF.loc[hemodynamicKeptDF.groupby(['CPR', 'MeasurementTime', 'MeasurementName'])['Value'].idxmax()]

In [ ]:
cleanAnesthesiaEventDF = pd.read_csv(r'FilteredData\CleanAnesthesiaTime.csv')
cleanAnesthesiaEventDF = cleanAnesthesiaEventDF.drop(columns = ['FirstEvent' , 'LastEvent' , 'StartOfProcedure' , 'EndOfProcedure' , 'TimeInMinutes'])
cleanAnesthesiaEventDF = cleanAnesthesiaEventDF.sort_values(by = ['CPR' , 'EventTime' , 'Event'])
cleanAnesthesiaEventDF['EventTime'] = pd.to_datetime(cleanAnesthesiaEventDF['EventTime'])

hemodynamicKeptDF['MeasurementTime'] = pd.to_datetime(hemodynamicKeptDF['MeasurementTime'])

duplicates = cleanAnesthesiaEventDF[cleanAnesthesiaEventDF.duplicated(subset=['CPR', 'Event'], keep=False)]

In [ ]:
cleanAnesthesiaEventPivotDF = cleanAnesthesiaEventDF.pivot(index='CPR', columns='Event', values='EventTime').reset_index()
hemodynamicKeptMergedDF = pd.merge(hemodynamicKeptDF, cleanAnesthesiaEventPivotDF, on='CPR')

In [ ]:
phase1 = hemodynamicKeptMergedDF[(hemodynamicKeptMergedDF['MeasurementTime'] >= hemodynamicKeptMergedDF['Induktion']) & (hemodynamicKeptMergedDF['MeasurementTime'] < hemodynamicKeptMergedDF['Aorta tang på'])]
phase2 = hemodynamicKeptMergedDF[(hemodynamicKeptMergedDF['MeasurementTime'] >= hemodynamicKeptMergedDF['Aorta tang på']) & (hemodynamicKeptMergedDF['MeasurementTime'] < hemodynamicKeptMergedDF['Aorta tang af'])]
phase3 = hemodynamicKeptMergedDF[(hemodynamicKeptMergedDF['MeasurementTime'] >= hemodynamicKeptMergedDF['Aorta tang af']) & (hemodynamicKeptMergedDF['MeasurementTime'] < hemodynamicKeptMergedDF['Stop Data Indsamling'])]
phase1 = phase1.drop(columns = ['Induktion' , 'Aorta tang på' , 'Aorta tang af' , 'Stop Data Indsamling'])
phase2 = phase2.drop(columns = ['Induktion' , 'Aorta tang på' , 'Aorta tang af' , 'Stop Data Indsamling'])
phase3 = phase3.drop(columns = ['Induktion' , 'Aorta tang på' , 'Aorta tang af' , 'Stop Data Indsamling'])

In [ ]:
phase1['CPR'] = phase1['CPR'].astype('category')
phase1['MeasurementName'] = phase1['MeasurementName'].astype('category')
phase1['Value'] = pd.to_numeric(phase1['Value'], downcast='float')

phase2['CPR'] = phase2['CPR'].astype('category')
phase2['MeasurementName'] = phase2['MeasurementName'].astype('category')
phase2['Value'] = pd.to_numeric(phase2['Value'], downcast='float')

phase3['CPR'] = phase3['CPR'].astype('category')
phase3['MeasurementName'] = phase3['MeasurementName'].astype('category')
phase3['Value'] = pd.to_numeric(phase3['Value'], downcast='float')

In [ ]:
phase1Percentiles = phase1[phase1['MeasurementName'].isin(['MAP', 'CVP' , 'Pulse'])].groupby(['CPR', 'MeasurementName'])['Value'].agg(
    p5 =lambda x: x.quantile(0.05),
    p10=lambda x: x.quantile(0.10),
    p25=lambda x: x.quantile(0.25),
    p50=lambda x: x.quantile(0.50),
    p75=lambda x: x.quantile(0.75),
    p90=lambda x: x.quantile(0.90),
    p95=lambda x: x.quantile(0.95)
).reset_index()

phase1Percentiles = phase1Percentiles[(phase1Percentiles['MeasurementName'] == 'CVP') | (phase1Percentiles['MeasurementName'] == 'MAP') | (phase1Percentiles['MeasurementName'] == 'Pulse')]
phase1PercentilesPivot = phase1Percentiles.pivot(index='CPR', columns='MeasurementName')
phase1PercentilesPivot.columns = [f'{col[1]}_{col[0]}' for col in phase1PercentilesPivot.columns]

phase1OneHot = pd.get_dummies(phase1[phase1['MeasurementName'].isin(['LAP', 'PAP'])].set_index('CPR')['MeasurementName']).groupby('CPR').max().reset_index()
phase1OneHot = phase1OneHot.drop(columns = ['MAP', 'CVP' , 'Pulse'])

phase1FinalHemoDF = phase1PercentilesPivot.merge(phase1OneHot, on='CPR', how='left')
phase1FinalHemoDF.columns = ['CPR'] + [f'{col}_p{p}' for p in [5, 10, 25, 50, 75, 90, 95] for col in ['CVP' , 'MAP', 'Pulse']] + ['LAP', 'PAP']
phase1FinalHemoDF['LAP'] = phase1FinalHemoDF['LAP'].fillna(0)
phase1FinalHemoDF['PAP'] = phase1FinalHemoDF['PAP'].fillna(0)
phase1FinalHemoDF = phase1FinalHemoDF.dropna()

In [ ]:
phase2Percentiles = phase2[phase2['MeasurementName'].isin(['MAP', 'CVP'])].groupby(['CPR', 'MeasurementName'])['Value'].agg(
    p5 =lambda x: x.quantile(0.05),
    p10=lambda x: x.quantile(0.10),
    p25=lambda x: x.quantile(0.25),
    p50=lambda x: x.quantile(0.50),
    p75=lambda x: x.quantile(0.75),
    p90=lambda x: x.quantile(0.90),
    p95=lambda x: x.quantile(0.95)
).reset_index()

phase2Percentiles = phase2Percentiles[(phase2Percentiles['MeasurementName'] == 'CVP') | (phase2Percentiles['MeasurementName'] == 'MAP')]
phase2PercentilesPivot = phase2Percentiles.pivot(index='CPR', columns='MeasurementName')
phase2PercentilesPivot.columns = [f'{col[1]}_{col[0]}' for col in phase2PercentilesPivot.columns]

phase2OneHot = pd.get_dummies(phase2[phase2['MeasurementName'].isin(['LAP', 'PAP'])].set_index('CPR')['MeasurementName']).groupby('CPR').max().reset_index()
phase2OneHot = phase2OneHot.drop(columns = ['MAP', 'CVP' , 'Pulse'])

phase2FinalHemoDF = phase2PercentilesPivot.merge(phase2OneHot, on='CPR', how='left')
phase2FinalHemoDF.columns = ['CPR'] + [f'{col}_p{p}' for p in [5, 10, 25, 50, 75, 90, 95] for col in ['CVP' , 'MAP']] + ['LAP', 'PAP']
phase2FinalHemoDF['LAP'] = phase2FinalHemoDF['LAP'].fillna(0)
phase2FinalHemoDF['PAP'] = phase2FinalHemoDF['PAP'].fillna(0)
phase2FinalHemoDF = phase2FinalHemoDF.dropna()

In [ ]:
phase3Percentiles = phase3[phase3['MeasurementName'].isin(['MAP', 'CVP' , 'Pulse'])].groupby(['CPR', 'MeasurementName'])['Value'].agg(
    p5 =lambda x: x.quantile(0.05),
    p10=lambda x: x.quantile(0.10),
    p25=lambda x: x.quantile(0.25),
    p50=lambda x: x.quantile(0.50),
    p75=lambda x: x.quantile(0.75),
    p90=lambda x: x.quantile(0.90),
    p95=lambda x: x.quantile(0.95)
).reset_index()

phase3Percentiles = phase3Percentiles[(phase3Percentiles['MeasurementName'] == 'CVP') | (phase3Percentiles['MeasurementName'] == 'MAP') | (phase3Percentiles['MeasurementName'] == 'Pulse')]
phase3PercentilesPivot = phase3Percentiles.pivot(index='CPR', columns='MeasurementName')
phase3PercentilesPivot.columns = [f'{col[1]}_{col[0]}' for col in phase3PercentilesPivot.columns]

phase3OneHot = pd.get_dummies(phase3[phase3['MeasurementName'].isin(['LAP', 'PAP'])].set_index('CPR')['MeasurementName']).groupby('CPR').max().reset_index()
phase3OneHot = phase3OneHot.drop(columns = ['MAP', 'CVP' , 'Pulse'])

phase3FinalHemoDF = phase3PercentilesPivot.merge(phase3OneHot, on='CPR', how='left')
phase3FinalHemoDF.columns = ['CPR'] + [f'{col}_p{p}' for p in [5, 10, 25, 50, 75, 90, 95] for col in ['CVP' , 'MAP', 'Pulse'] ] + ['LAP', 'PAP']
phase3FinalHemoDF['LAP'] = phase3FinalHemoDF['LAP'].fillna(0)
phase3FinalHemoDF['PAP'] = phase3FinalHemoDF['PAP'].fillna(0)
phase3FinalHemoDF = phase3FinalHemoDF.dropna()

In [ ]:
print(len(list(set(phase1FinalHemoDF['CPR']))))
print(len(list(set(phase2FinalHemoDF['CPR']))))
print(len(list(set(phase3FinalHemoDF['CPR']))))

In [ ]:
phase1FinalHemoDF.to_csv(r'FilteredData\phase1FinalHemo.csv' , index = False)
phase2FinalHemoDF.to_csv(r'FilteredData\phase2FinalHemo.csv' , index = False)
phase3FinalHemoDF.to_csv(r'FilteredData\phase3FinalHemo.csv' , index = False)

In [ ]:
print(len(list(set(hemodynamicKeptDF['CPR']))))

In [ ]:
check = hemodynamicKeptDF['MeasurementName'].value_counts()

In [ ]:
hemodynamicDF.to_csv(r'FilteredData\HemodynamicMeasuresRAW.csv')